<a href="https://colab.research.google.com/github/diego-oyarce/trabajo-final-ml/blob/main/trabajo_final_ML_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba de ciencia de datos de Empresa SoftWorld

Primero se procede a importar las librerias necesarias para trabajar con los datos y realizar el modelo de aprendizaje automático.

In [153]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

Una vez importadas las librerias y funciones se continua con la lectura de archivos. Para esto se utiliza pandas.

In [154]:
orders = pd.read_csv('orders.csv')
order_products = pd.read_csv('order_products.csv')
shoppers = pd.read_csv('shoppers.csv')
store_branch = pd.read_csv('storebranch.csv')

Se renombran los campos de lan y lng de los dataset orders y storebranch para no tener problemas en la fusión.

In [155]:
orders.rename(columns={'lat': 'order_lat', 'lng': 'order_lng'}, inplace=True)
store_branch.rename(columns={'lat': 'branch_lat', 'lng': 'branch_lng'}, inplace=True)

A continuación, se unen los conjuntos de datos relevantes haciendo uso de las columnas clave.

In [156]:
data = orders.merge(order_products, on='order_id')
data = data.merge(shoppers, on='shopper_id')
data = data.merge(store_branch, on='store_branch_id')

Se codifican las variables categóricas, en este caso solo 'buy_unit'

In [157]:
data = pd.get_dummies(data, columns=['buy_unit', 'on_demand'])

Ahora se transforma el campo 'promised_time' a valores numéricos que se puedan usar en el modelo.

In [158]:
data['promised_time'] = pd.to_datetime(data['promised_time'])
data['promised_day'] = data['promised_time'].dt.day
data['promised_month'] = data['promised_time'].dt.month
data['promised_dayofweek'] = data['promised_time'].dt.dayofweek
data['promised_hour'] = data['promised_time'].dt.hour

Para continuar, se eliminan las filas con datos faltantes.

In [159]:
data.dropna(inplace=True)

Se seleccionan las características relevantes para el modelo.

In [160]:
features = ['quantity', 'found_rate', 'picking_speed', 'accepted_rate', 'rating',
            'buy_unit_KG', 'buy_unit_UN', 'on_demand_False', 'on_demand_True',
            'order_lat', 'order_lng', 'promised_day', 'promised_month', 'promised_dayofweek', 'promised_hour',
            'branch_lat', 'branch_lng']
target = 'total_minutes'

Se continúa dividiendo los datos en características y variable objetivo.

In [161]:
X = data[features]
y = data[target]

Ahora se dividen los datos en conjuntos aleatorios de prueba y entrenamiento.

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

A continuación se crea y entrena un modelo Random Forest para realizar las predicciones.

In [164]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

Ahora se realizan predicciones utilizando el conjunto de prueba.

In [165]:
y_pred = model.predict(X_test)

Ahora se evalua el rendimiento del modelo con mean squared error.

In [166]:
mse = mean_squared_error(y_test, y_pred)
print('Mean Squared Error:', mse)

Mean Squared Error: 8.009541149889465
